# Task 2: General Hartree-Fock solver

Having set up an initial single-particle basis set $\{\chi_{\alpha}\}_{\alpha = 1}^{l}$ we now set out to find the ground state of the two-body fermion problem.
The full time-independent Hamiltonian used by {cite}`zanghellini_2004` is given by
\begin{align}
    \hat{H}
    = \hat{h} + \hat{u}
    = \sum_{i = 1}^{n}\left(
        -\frac{1}{2}\frac{\text{d}^2}{\text{d}x^2}
        + \frac{1}{2}\omega^2 x^2
    \right)
    + \sum_{i = 1}^{n} \sum_{j = i + 1}^{n} \frac{\alpha}{\sqrt{(x_i - x_j)^2 + a^2}},
\end{align}
where the number of particles $n = 2$, $a = 0.25$, $\alpha = 1$ (not to be confused with the index $\alpha$ in the basis of harmonic oscillator eigenstates), and $\omega = 0.25$.

We start from the time-independent Schrödinger equation
\begin{align}
    \hat{H} | \Psi \rangle = E | \Psi \rangle,
\end{align}
where $| \Psi \rangle$ is a many-body wave function.
We know that in the non-interacting case the exact ground state of the Hamiltonian will be a single Slater determinant (which we denote $| \Phi \rangle$) of the lowest single-particle eigenstates.
This motivates the ansatz of using a single Slater determinant for the full many-body problem
\begin{align}
    | \Psi \rangle \approx | \Phi \rangle
    = | \phi_1 \phi_2 \dots \phi_n \rangle,
\end{align}
where the $n$ lowest _molecular orbitals_ (MO) (also known as the Hartree-Fock orbitals) from $\{\phi_p\}_{p = 1}^{k}$ ($k \leq n$) are the primary unknowns.
```{note}
We use a convention where split up the set of molecular orbitals into a set of _occupied_ and _virtual_ orbitals.
That is
\begin{align}
    \{ \phi_p \}_{p = 1}^{k}
    = \{ \phi_i \}_{i = 1}^{n} \cup \{ \phi_a \}_{a = n + 1}^{k},
\end{align}
where we let the indices $i, j, k, l, \dots \in \{1, \dots, n\}$ denote the occupied orbitals, i.e., the orbitals that are contained in the Hartree-Fock Slater determinant, $a, b, c, d, \dots \in \{n + 1, \dots, k \}$ are the indices of the virtual orbitals that are not contained in the reference determinant, and $p, q, r, s, \dots \in \{1, \dots, k\}$ the full set of orbitals.
```
The Hartree-Fock method method finds the Slater determinant that minimizes the energy, i.e., gets closest to the "true" many-body ground state energy $E_{gs}$ of the problem using the variational principle
\begin{align}
    E_{gs} \leq E[\Phi, \Phi^{*}] = E[\phi_1, \dots, \phi_n] = \langle \Phi | \hat{H} | \Phi \rangle,
\end{align}
subject to the constraint that the molecular orbitals are orthonormal, $\langle \phi_i | \phi_j \rangle = \delta_{ij}$ ($\implies \langle \Phi | \Phi \rangle = 1$).
To include this requirement in the variational principle we use Lagrange's method of undetermined multipliers

$$
    L[\phi_1, \dots, \phi_n, \boldsymbol{\lambda}]
    = E[\phi_1, \dots, \phi_n]
    - \lambda_{qp} \left(
        \langle \phi_p | \phi_q \rangle
        - \delta_{pq}
    \right),
$$ (eq:hf-lagrangian)
where $\boldsymbol{\lambda}$ are the Lagrange multipliers.

```{note}
Add derivation of the (canonical) GHF-equations.
```

Minimization of the Lagrangian in {eq}`eq:hf-lagrangian` yields the Hartree-Fock eigenvalue equation

$$
    \hat{f} | \phi_p \rangle = \varepsilon_p | \phi_p \rangle,
$$ (eq:canonical-hf-equation)
where $\hat{f}$ is the single-particle Fock-operator.
Note that this equation is not limited to the occupied states, but to all the molecular orbitals.
The matrix elements of the single-particle Fock operator are given by
\begin{align}
    \langle \psi | \hat{f} | \xi \rangle
    = \langle \psi | \hat{h} | \xi \rangle
    + \sum_{i = 1}^{n} \langle \psi \phi_i | \hat{u} | \xi \phi_i \rangle
    - \sum_{i = 1}^{n} \langle \psi \phi_i | \hat{u} | \phi_i \xi \rangle
    = \langle \psi | \hat{h} | \xi \rangle
    + \sum_{i = 1}^{n} \langle \psi \phi_i | \hat{u} | \xi \phi_i \rangle_{AS},
\end{align}
where $|\psi\rangle$ and $|\xi \rangle$ are two arbitrary single-particle states.

## The Roothan-Hall equations

The way we solve the Hartree-Fock equation in {eq}`eq:canonical-hf-equation` is by expanding the Hartree-Fock orbitals $\{ \phi_p \}_{p = 1}^k$ in an atomic orbital basis $\{ \psi_{\mu} \}_{\mu = 1}^{l}$ (which we here take to be the orthonormal harmonic oscillator basis states with spin-doubling), and then projecting the equations onto the known basis set.
```{note}
In general $k \leq l$ for the molecular orbitals.
The consequence of this is that Hartree-Fock can be used as a method to truncate the basis set.
This is common when using computationally more complex post Hartree-Fock methods such as e.g., coupled-cluster, to lower simulation times.
However, in this project we limit our attention to $k = l$.
```
We express a basis transformation from the AO basis to the MO basis by

$$
    |\phi_p\rangle = \sum_{\mu = 1}^{l} C_{\mu p} | \psi_{\mu} \rangle,
$$ (eq:basis-transformation)
where the coefficients $C_{\mu p}$ now become our primary unknowns in the Hartree-Fock method.
Projecting the AO basis onto equation {eq}`eq:canonical-hf-equation` we have
\begin{gather}
    \langle \psi_{\mu} | \hat{f} | \phi_p \rangle
    = \varepsilon_{p} \langle \psi_{\mu} | \phi_p \rangle
    \implies
    \sum_{\nu = 1}^{l} f_{\mu \nu} C_{\nu p}
    = \varepsilon_{p} \sum_{\nu = 1}^{l} \delta_{\mu \nu} C_{\nu p}
    =  C_{\mu p} \varepsilon_{p}
    \\
    \implies
    \mathbf{F} \mathbf{C}
    = \mathbf{C} \boldsymbol{\varepsilon},
\end{gather}
where $[\mathbf{F}]_{\mu \nu} = f_{\mu \nu} = \langle \psi_{\mu} | \hat{f} | \psi_{\nu} \rangle$ are the Fock matrix elements in the AO basis, $[\mathbf{C}]_{\mu p} = C_{\mu p}$ the basis transformation coefficient matrix, and $\boldsymbol{\varepsilon} = [\varepsilon_1, \dots, \varepsilon_k]^T$ the vector of Hartree-Fock single-particle eigenenergies (not to be confused with the full many-body energy).
The last eigenvalue equation is known as the Roothan-Hall equations.
Here both $\mathbf{C}$ and $\boldsymbol{\varepsilon}$ are the primary unknowns.
We now need to construct the Fock matrix in the atomic orbital basis, that is
\begin{align}
    f_{\mu \nu}
    &= \langle \psi_{\mu} | \hat{f} | \psi_{\nu} \rangle
    = \langle \psi_{\mu} | \hat{h} | \psi_{\nu} \rangle
    + \sum_{i = 1}^{n}
    \langle \psi_{\mu} \phi_i | \hat{u} | \psi_{\nu} \phi_i \rangle_{AS}.
\end{align}
Replacing the sum over the occupied molecular orbitals in the two-body elements by equation {eq}`eq:basis-transformation` we get
\begin{align}
    f_{\mu \nu}
    &= \langle \psi_{\mu} | \hat{h} | \psi_{\nu} \rangle
    + \sum_{i = 1}^{n}
    C^{*}_{\kappa i} C_{\lambda i}
    \langle \psi_{\mu} \psi_{\kappa} | \hat{u} | \psi_{\nu} \psi_{\lambda} \rangle_{AS}
    = h_{\mu \nu}
    + D_{\lambda \kappa}
    u^{\mu \kappa}_{\nu \lambda},
\end{align}
where we have defined the density matrix $D_{\lambda \kappa} = \sum_{i = 1}^{n} C^{*}_{\kappa i} C_{\lambda i}$.

A complicating factor with the Roothan-Hall equations is that the Fock matrix $\mathbf{F}$ depends on the coefficient matrix $\mathbf{C}$.
To get around this we use a technique called self-consistent field iterations.